In [ ]:
#Mouting The Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.applications import vgg16
img_rows, img_cols = 64, 64
VGG = vgg16.VGG16(weights='imagenet',include_top=False ,input_shape=(img_rows, img_cols,3))

In [ ]:
#Freezing The Pre-trained Layers
for layer in VGG.layers:
  layer.trainable = False

In [ ]:
#Defining Layers and Placing new Layers 
def top(bottom_model, num_classes):
  top_model = bottom_model.output
  top_model = GlobalAveragePooling2D()(top_model)
  top_model = Dense(1024,activation = 'relu')(top_model)
  top_model = Dense(512, activation = 'relu')(top_model)
  top_model = Dense(512, activation = 'relu')(top_model)
  top_model = Dense(256, activation = 'relu')(top_model)
  top_model = Dense(128, activation = 'relu')(top_model)
  top_model = Dense(num_classes, activation = 'softmax')(top_model)
  return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D,  MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model

num_classes = 3

Model_Use = top(VGG, num_classes)

model = Model(inputs = VGG.input, outputs = Model_Use)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

Train_Path = '/content/drive/MyDrive/1918026_hirdayanshsah_miniproject_5sem2021/train'
Validation_Path = '/content/drive/MyDrive/1918026_hirdayanshsah_miniproject_5sem2021/Validation'

# generating More Images from the Dataset for Increasing 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range=45,
                                  width_shift_range=0.3,
                                  height_shift_range=0.3,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255,
                                        rotation_range = 45,
                                        width_shift_range = 0.3,
                                        height_shift_range = 0.3,
                                        horizontal_flip = True,
                                        fill_mode = 'nearest')

batch = 64 # Number of Photo picked per epoch 

train_generator = train_datagen.flow_from_directory(Train_Path,
                                                    target_size = (img_rows,img_cols),
                                                    batch_size = batch,
                                                    class_mode = 'categorical')

validation_generator = validation_datagen.flow_from_directory(
    Validation_Path,
    target_size = (img_rows, img_cols),
    batch_size = batch,
    class_mode = 'categorical'
)

Found 5232 images belonging to 3 classes.
Found 2766 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("/content/drive/MyDrive/1918026_hirdayanshsah_miniproject_5sem2021/Detect_vgg.h5",
                             monitor = "val_loss",
                             mode = "min",
                             save_best_only=True,
                             verbose = 1)



model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(learning_rate = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 5232
nb_validation_samples = 2766

epochs = 10
batch_size = 64

# //-> Represents The Floor Division

history = model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = checkpoint,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size
)

Epoch 1/10
81/81 [==============================] - ETA: 0s - loss: 0.2688 - accuracy: 0.9106 
Epoch 00001: val_loss improved from inf to 0.08796, saving model to /content/drive/MyDrive/1918026_hirdayanshsah_miniproject_5sem2021/Detect_vgg.h5
81/81 [==============================] - 3314s 41s/step - loss: 0.2688 - accuracy: 0.9106 - val_loss: 0.0880 - val_accuracy: 0.9662
Epoch 2/10
81/81 [==============================] - ETA: 0s - loss: 0.1580 - accuracy: 0.9452
Epoch 00002: val_loss did not improve from 0.08796
81/81 [==============================] - 27s 338ms/step - loss: 0.1580 - accuracy: 0.9452 - val_loss: 0.1598 - val_accuracy: 0.9506
Epoch 3/10
81/81 [==============================] - ETA: 0s - loss: 0.1340 - accuracy: 0.9541
Epoch 00003: val_loss did not improve from 0.08796
81/81 [==============================] - 28s 346ms/step - loss: 0.1340 - accuracy: 0.9541 - val_loss: 0.2807 - val_accuracy: 0.9299
Epoch 4/10
81/81 [==============================] - ETA: 0s - loss: 0.1